In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv('./data/raw_data/spx_eod_202201.csv', skipinitialspace=True)

In [25]:
def process_options(df_opt, call = True):
    """Cleans up column names and add time to live (Ttl) and volatility column to the dataframe"""
    keys = {key: key[key.find("[")+1:key.find("]")][0] + key[key.find("[")+1:key.find("]")][1:].lower()  for key in df_opt.keys()}
    df_opt = df_opt.rename(columns=keys)

    if call:
        keys = {"C_ask": "Ask", "C_bid": "Bid"}
    else:
        keys = {"P_ask": "Ask", "P_bid": "Bid"}
    df_opt = df_opt.rename(columns=keys)

    df_opt["Quote_date"] = pd.to_datetime(df_opt["Quote_date"])
    df_opt["Expire_date"] = pd.to_datetime(df_opt["Expire_date"])
    df_opt["TTM"] = df_opt.apply(lambda row: (row.Expire_date - row.Quote_date).days, axis = 1)
    df_opt["Price"] = (df_opt["Ask"] + df_opt["Bid"])/2

    #df_opt["Moneyness"] = df_opt["Underlying_last"] / df_opt["Strike"]
    #df_opt["Bid_strike"] = df_opt["Bid"] / df_opt["Strike"]
    #df_opt["Ask_strike"] = df_opt["Ask"] / df_opt["Strike"]
     

    #columns = ["Quote_date", "Expire_date",  "Underlying_last", "Strike", "Ask", "Bid",  "Bid_strike", "Ask_strike", "Moneyness", "Ttl", "Volatility"]
    #columns = ["Quote_date", "Expire_date", "Ask", "Bid", "Underlying_last", "Strike", "Ttl", "Volatility"]
    columns = ["Quote_date", "Expire_date", "Price", "Underlying_last", "Strike", "TTM"]
    df_opt = df_opt[columns]
    df_opt = df_opt[(df_opt["TTM"] != 0) & (df_opt["TTM"] <= 365*3)]
    return df_opt[columns]

In [26]:
df = process_options(df)
N_TIMESTEPS = 20


In [ ]:
padded = np.insert(underlying.close.values, 0, np.array([np.nan] * N_TIMESTEPS))
rolled = np.column_stack([np.roll(padded, i) for i in range(N_TIMESTEPS)])
rolled = rolled[~np.isnan(rolled).any(axis=1)]
rolled = np.column_stack((underlying.date.values[N_TIMESTEPS - 1:], rolled))
price_history = pd.DataFrame(data=rolled)
joined = df.join(price_history.set_index(0), on='date')
call_df = joined[joined.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = joined[joined.cp_flag == 'P'].drop(['cp_flag'], axis=1)
call_df = call_df.drop(columns=['date'])
put_df = put_df.drop(columns=['date'])
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1).values,
                                                                        ((call_df.best_bid + call_df.best_offer) / 2).values,
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1).values,
                                                                    ((put_df.best_bid + put_df.best_offer) / 2).values,
                                                                    test_size=0.01, random_state=42)
call_X_train = [call_X_train[:, -N_TIMESTEPS:].reshape(call_X_train.shape[0], N_TIMESTEPS, 1), call_X_train[:, :4]]
call_X_test = [call_X_test[:, -N_TIMESTEPS:].reshape(call_X_test.shape[0], N_TIMESTEPS, 1), call_X_test[:, :4]]
put_X_train = [put_X_train[:, -N_TIMESTEPS:].reshape(put_X_train.shape[0], N_TIMESTEPS, 1), put_X_train[:, :4]]
put_X_test = [put_X_test[:, -N_TIMESTEPS:].reshape(put_X_test.shape[0], N_TIMESTEPS, 1), put_X_test[:, :4]]